# Data Preparation & Exploration for Climate (ulmo)

I want to merge the data from GSOD to my ILI data by getting a summary of the data by position
-  exploring daymet : daily temperature, precipitation for any locationsion in the US [reference](https://github.com/ulmo-dev/ulmo/blob/master/examples/Using%20Daymet%20weather%20data%20from%20ORNL%20webservice.ipynb)

In [1]:
# import packages and modules
from ulmo.nasa import daymet
from delphi_epidata import Epidata

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle

pd.set_option('display.max_columns', 100)

/anaconda/lib/python3.6/site-packages/ulmo/twc/kbdi/core.py:20: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  CSV_SWITCHOVER = pandas.tslib.Timestamp('2016-10-01')


In [7]:
#test
ili_test = pd.read_csv("../data/raw.csv")
ili_test.head()

,statename,activity_level,activity_level_label,weekend,season,weeknumber,Latitude,Longitude
0,Alabama,1,Minimal,Oct-01-2016,2015-16,39,32.806671,-86.79113
1,Alabama,1,Minimal,Sep-24-2016,2015-16,38,32.806671,-86.79113
2,Alabama,1,Minimal,Sep-17-2016,2015-16,37,32.806671,-86.79113
3,Alabama,1,Minimal,Sep-10-2016,2015-16,36,32.806671,-86.79113
4,Alabama,1,Minimal,Sep-03-2016,2015-16,35,32.806671,-86.79113


In [39]:
coordinate = ili_test[['statename','Latitude','Longitude']]
coordinate = coordinate.drop_duplicates()
coordinate = coordinate.reset_index(drop=True)
coordinate.head()
#print("the data contains {} rows".format(len(coordinate)))

,statename,Latitude,Longitude
0,Alabama,32.806671,-86.791130
1,Alaska,61.370716,-152.404419
2,Arizona,33.729759,-111.431221
3,Arkansas,34.969704,-92.373123
4,California,36.116203,-119.681564


In [123]:
ili_test.shape

(15933, 8)

-  The latitude (WGS84), value between 52.0 and 14.5.
-  The longitude (WGS84), value between -131.0 and -53.0.

In [43]:
coordinate = coordinate[coordinate.Longitude >= -131.0]
coordinate = coordinate[coordinate.Longitude <= -53.0]
coordinate = coordinate[coordinate.Latitude >= 14.5]
coordinate = coordinate[coordinate.Latitude <= 52.0]
coordinate = coordinate.reset_index(drop=True)
coordinate


,statename,Latitude,Longitude
0,Alabama,32.806671,-86.791130
1,Arizona,33.729759,-111.431221
2,Arkansas,34.969704,-92.373123
3,California,36.116203,-119.681564
4,Colorado,39.059811,-105.311104
5,Connecticut,41.597782,-72.755371
6,Delaware,39.318523,-75.507141
7,District of Columbia,38.897438,-77.026817
8,Florida,27.766279,-81.686783
9,Georgia,33.040619,-83.643074


In [89]:
climate = []
for i in range(len(coordinate.statename)):
    df = daymet.get_daymet_singlepixel(longitude=coordinate.Longitude[i], latitude=coordinate.Latitude[i], 
                                   years=[2010,2015])
    df['statename'] = coordinate.statename[i]
    df['Latitude'] = coordinate.Latitude[i]
    df['Longitude'] = coordinate.Longitude[i]
    climate.append(df)
    
climate = pd.concat(climate)

climate['year'] = climate.index.year
climate['month'] = climate.index.month
climate['day'] = climate.index.day

climate.head()

making request for latitude, longitude: 32.806671, -86.79113000000001
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=32.806671&lon=-86.79113000000001&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=32.806671&lon=-86.79113&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 33.729759, -111.431221
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=33.729759&lon=-111.431221&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=33.729759&lon=-111.431221&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 34.969704, -92.37312299999999
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=34.969704&lon=-92.37312299999999&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=34.969704&lon=-92.373123&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 36.116203000000006, -119.681564
processing data from request: https://daymet.ornl.gov/

making request for latitude, longitude: 34.840515, -106.24848200000001
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=34.840515&lon=-106.24848200000001&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=34.840515&lon=-106.248482&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 42.165726, -74.948051
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=42.165726&lon=-74.948051&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=42.165726&lon=-74.948051&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 35.630066, -79.80641899999999
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=35.630066&lon=-79.80641899999999&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=35.630066&lon=-79.806419&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 47.528912, -99.784012
processing data from request: https://daymet.ornl.gov/data/send

,year,yday,prcp,tmax,tmin,statename,Latitude,Longitude
2010-01-01,2010,1,1.0,10.5,1.0,Alabama,32.806671,-86.79113
2010-01-02,2010,2,0.0,7.5,-3.0,Alabama,32.806671,-86.79113
2010-01-03,2010,3,0.0,4.5,-4.5,Alabama,32.806671,-86.79113
2010-01-04,2010,4,0.0,2.5,-5.5,Alabama,32.806671,-86.79113
2010-01-05,2010,5,0.0,1.5,-8.5,Alabama,32.806671,-86.79113
2010-01-06,2010,6,0.0,4.0,-9.0,Alabama,32.806671,-86.79113
2010-01-07,2010,7,0.0,5.0,-7.0,Alabama,32.806671,-86.79113
2010-01-08,2010,8,0.0,1.5,-7.5,Alabama,32.806671,-86.79113
2010-01-09,2010,9,0.0,0.0,-9.0,Alabama,32.806671,-86.79113
2010-01-10,2010,10,0.0,2.0,-9.0,Alabama,32.806671,-86.79113


In [46]:
# save the file
climate.to_csv("../data/climate.csv", sep='\t', encoding='utf-8')

In [47]:
res = Epidata.fluview(['nat'], [201440, Epidata.range(201501, 201510)])
print(res['result'], res['message'], len(res['epidata']))

1 success 11


In [48]:
#test
#ornl_lat, ornl_long = 35.9313167, -84.3104124
#df = daymet.get_daymet_singlepixel(longitude=ornl_long, latitude=ornl_lat, 
#                                   years=[2012,2013])


In [49]:
#df.index.year

In [90]:
climate['year'] = climate.index.year
climate['month'] = climate.index.month
climate['day'] = climate.index.day
#df.drop('index', axis=0, inplace=True)
climate.head()

,year,yday,prcp,tmax,tmin,statename,Latitude,Longitude,month,day
2010-01-01,2010,1,1.0,10.5,1.0,Alabama,32.806671,-86.79113,1,1
2010-01-02,2010,2,0.0,7.5,-3.0,Alabama,32.806671,-86.79113,1,2
2010-01-03,2010,3,0.0,4.5,-4.5,Alabama,32.806671,-86.79113,1,3
2010-01-04,2010,4,0.0,2.5,-5.5,Alabama,32.806671,-86.79113,1,4
2010-01-05,2010,5,0.0,1.5,-8.5,Alabama,32.806671,-86.79113,1,5


[Pandas dataframe groupeby datetime month](https://stackoverflow.com/questions/24082784/pandas-dataframe-groupby-datetime-month)


In [91]:
climate.shape

(35770, 10)

In [143]:
df_month = climate[['month','year', 'prcp', 'tmax', 'tmin','statename']].groupby(['statename','year', 'month',], as_index = False).mean()
df_month = df_month.rename(columns={'yday':'day',
                                    'prcp':'mean_prcp',
                                   'tmax': 'mean_tmax',
                                   'tmin':'mean_tmin'})
df_month.head()

,statename,year,month,mean_prcp,mean_tmax,mean_tmin
0,Alabama,2010,1,6.354839,10.758065,-1.258065
1,Alabama,2010,2,4.964286,10.892857,-1.017857
2,Alabama,2010,3,5.516129,17.177419,4.419355
3,Alabama,2010,4,3.033333,26.033333,9.783333
4,Alabama,2010,5,4.612903,29.322581,17.096774


In [93]:
df_month.shape

(1176, 9)

We have :
-  categorical variable : year, yday(integer)
-  other variable : floats or interger
-  index : datetime

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 730 entries, 2012-01-01 to 2013-12-31
Data columns (total 9 columns):
year         730 non-null int64
yday         730 non-null int64
prcp         730 non-null float64
tmax         730 non-null float64
tmin         730 non-null float64
Latitude     730 non-null float64
Longitude    730 non-null float64
month        730 non-null int64
day          730 non-null int64
dtypes: float64(5), int64(4)
memory usage: 57.0 KB


In [95]:
#load ILI data from csv
ili = pd.read_csv("../data/raw.csv")
ili.head()

,statename,activity_level,activity_level_label,weekend,season,weeknumber,Latitude,Longitude
0,Alabama,1,Minimal,Oct-01-2016,2015-16,39,32.806671,-86.79113
1,Alabama,1,Minimal,Sep-24-2016,2015-16,38,32.806671,-86.79113
2,Alabama,1,Minimal,Sep-17-2016,2015-16,37,32.806671,-86.79113
3,Alabama,1,Minimal,Sep-10-2016,2015-16,36,32.806671,-86.79113
4,Alabama,1,Minimal,Sep-03-2016,2015-16,35,32.806671,-86.79113


In [96]:
# Clean the data 
# tramsform to datetime
ili['weekend'] = pd.to_datetime(ili['weekend'], format='%b-%d-%Y')
ili['season'] = pd.to_datetime(ili['season'], format='%Y-%y') 

# add year month and day in the data
ili['year'] = ili.weekend.dt.year
ili['month'] = ili.weekend.dt.month
ili['day'] = ili.weekend.dt.day

#remove data that we don't need
ili.drop(['weekend','season','weeknumber'], axis=1, inplace=True)

In [97]:
ili.head()

,statename,activity_level,activity_level_label,Latitude,Longitude,year,month,day
0,Alabama,1,Minimal,32.806671,-86.79113,2016,10,1
1,Alabama,1,Minimal,32.806671,-86.79113,2016,9,24
2,Alabama,1,Minimal,32.806671,-86.79113,2016,9,17
3,Alabama,1,Minimal,32.806671,-86.79113,2016,9,10
4,Alabama,1,Minimal,32.806671,-86.79113,2016,9,3


In [134]:
ili.shape

(15933, 8)

In [138]:
ili.drop_duplicates()
ili.shape

(15933, 8)

 ## Merge data

In [154]:
df = pd.merge(ili, df_month, on = ['statename', 'year', 'month' ])
df = df.sort_values(['statename','year','month'], ascending=True)
df = df.reset_index(drop=True)
df.to_csv("../data/train.csv", sep='\t', encoding='utf-8')
df.head()

,statename,activity_level,activity_level_label,Latitude,Longitude,year,month,day,mean_prcp,mean_tmax,mean_tmin
0,Alabama,1,Minimal,32.806671,-86.79113,2010,10,30,3.741935,26.870968,9.177419
1,Alabama,1,Minimal,32.806671,-86.79113,2010,10,23,3.741935,26.870968,9.177419
2,Alabama,1,Minimal,32.806671,-86.79113,2010,10,16,3.741935,26.870968,9.177419
3,Alabama,1,Minimal,32.806671,-86.79113,2010,10,9,3.741935,26.870968,9.177419
4,Alabama,1,Minimal,32.806671,-86.79113,2010,11,20,4.800000,19.933333,5.716667


In [137]:
df.shape

(3126, 14)

## Metadata

store meta-information about the variables in a DataFrame


-  **role**: response, explanatory (variable that we want to predict)
-  **level**: nominal, interval, ordinal, binary
-  **keep**: True or False
-  **dtype**: int, float, str

In [56]:
meta_ili = []
for f in ili.columns:
    # Defining the role
    if f == 'activity_level' or f == 'activity_level_label':
        role = 'response'
    else:
        role = 'explanatory'
         
    # Defining the level
    if 'statename' == f :
        level = 'nominal'
    elif 'activity_level_label' == f:
        level = 'ordinal'
    elif 'weekend' == f or 'season' == f or 'weeknumber' == f:
        level = 'interval'
    elif ili[f].dtype == float:
        level = 'ordinal'
    elif ili[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'activity_level_label':
        keep = False
    
    # Defining the data type 
    dtype = ili[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    meta_ili.append(f_dict)
    
meta = pd.DataFrame(meta_ili, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [57]:
meta           

,role,level,keep,dtype
varname,,,,
statename,explanatory,nominal,True,object
activity_level,response,ordinal,True,int64
activity_level_label,response,ordinal,False,object
weekend,explanatory,interval,True,datetime64[ns]
season,explanatory,interval,True,datetime64[ns]
weeknumber,explanatory,interval,True,int64
Latitude,explanatory,ordinal,True,float64
Longitude,explanatory,ordinal,True,float64


In [58]:
# Below the number of variables per role and level are displayed.

pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,explanatory,interval,3
1,explanatory,nominal,1
2,explanatory,ordinal,2
3,response,ordinal,2


### interval variables

In [59]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
ili[v].describe()

,activity_level,Latitude,Longitude
count,15933.000000,15933.000000,15933.000000
mean,1.957510,39.465562,-93.366505
std,2.183754,6.015489,19.102700
min,0.000000,21.094318,-157.498337
25%,1.000000,35.630066,-105.311104
50%,1.000000,39.849426,-89.616508
75%,2.000000,43.326618,-78.169968
max,10.000000,61.370716,-69.381927


In [60]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
ili[v].describe()

,statename
count,15933
unique,51
top,Alaska
freq,313


### Checking the cardinality of the categorical variables

In [62]:
v = meta[(meta.level == 'interval') & (meta.keep)].index

for f in v:
    dist_values = ili[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

Variable weekend has 313 distinct values
Variable season has 6 distinct values
Variable weeknumber has 53 distinct values
